In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [ ]:
timestamp = '20241009-144131'

fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath)
# display(df)

## Drop rows with nans.
df_deaths.dropna(inplace=True)
# display(df)

## Drop samples with negative inc deaths.
df_deaths = df_deaths[df_deaths.True_inc_deaths >= 0]
# display(df_deaths)

# # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # 

from epsampling.utils import drop_duplicate_cols

# timestamp = '20241009-143022' 

fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_normed_{timestamp}.csv')
df_acs = pd.read_csv(fpath,index_col='Fips')

# Remove duplicate columns
df_acs = drop_duplicate_cols(df_acs)

In [ ]:
df = df_deaths.merge(df_acs, on='Fips')
display(df)

In [ ]:
df['Target'] = df.apply(lambda x: x.True_inc_deaths / x.Pop * 100000, axis=1)
df['Naive'] = df.apply(lambda x: x.Naive_inc_deaths / x.Pop * 100000, axis=1)
df['State'] = df.apply(lambda x: x.Proj_inc_deaths / x.State_pop * 100000, axis=1)
df['Cumtm1'] = df.apply(lambda x: x.Cum_deaths_tm1 / x.Pop * 100000, axis=1)

In [ ]:
df = (df - df.min()) / (df.max() - df.min())

In [ ]:
df.columns[:15]

In [ ]:
df

In [ ]:
def get_chunks(lst, n):
    """Return successive n-sized chunks from lst."""
    chunks = []
    for i in range(0, len(lst), n):
        chunks.append(lst[i:i + n])
    return chunks

In [ ]:
chunks = get_chunks(list(df.Date.unique()), 4)

# idc = [2,10,18]
# idc = [10]

# for idx in idc:
    
idx = 18 #10

dates_test = chunks[idx]
dates_train = [x for x in df.Date.unique() if x not in dates_test]

df_train = df[df.Date.isin(dates_train)]
df_test = df[df.Date.isin(dates_test)]

feats = list(df_acs.columns) + ['State','Cumtm1','Naive',
                               'State_pop','Ratio','Pop']
print(feats)

X_train = df_train[feats]
X_test = df_test[feats]

y_train = df_train['Target']
y_test = df_test['Target']

y_naive = df_test['Naive']

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# train time for random split at 0.1 test_size? 2 min
clf = RandomForestRegressor(random_state=666, max_depth=6)
clf.fit(X_train, y_train)

In [ ]:
from epsampling.utils import get_performance

y_pred = clf.predict(X_test)

model_names = ['Deaths_pred', 'Deaths_naive']
model_preds = [y_pred, y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict